In [1]:
"""Load the primitive model and the DSL parse"""
from karanir.thanagor.model import *
from karanir.thanagor import KaranirThanagor, config

/Users/melkor/miniforge3/envs/Melkor/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Taichi] version 1.7.0, llvm 15.0.7, commit 7b58b0ff, osx, python 3.9.13


[I 04/30/24 18:17:17.148 967436] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
"""Define the domain specification for the demo test case"""
from karanir.thanagor.domain import load_domain_string
demo_domain_string = f"""
(domain scourge_demo)
(:type
    object - vector[float,100]
    position - vector[float,2]
    color - vector[float, 64]
    category
)
(:predicate
    color ?x-object -> vector[float,64]
    is-red ?x-object -> boolean
    is-blue ?x-object -> boolean
    is-ship ?x-object -> boolean
    is-house ?x-object -> boolean
)
(:derived
    is-green ?x-color expr: (??f ?x)
)
(:constraint
    (color: is-red is-blue)
    (category: is-ship is-house)
)
(:action
    (
        name:pick-up
        parameters:?x-object ?y
        precondition: (is-red (color ?x))
        effect: (
            if (is-blue ?y)
                (and
                (assign (is-blue ?x) 1.0 )
                (assign (is-red ?y) 0.0)
                )
            )
    )
)
"""
domain = load_domain_string(demo_domain_string)
domain.print_summary()

domain:
  scourge_demo
types:
  object - vector[float,['100']]
  position - vector[float,['2']]
  color - vector[float,['64']]
  category - object
predicates:
  color:['?x-object'] -> vector[float,['64']]
  is-red:['?x-object'] -> boolean
  is-blue:['?x-object'] -> boolean
  is-ship:['?x-object'] -> boolean
  is-house:['?x-object'] -> boolean
actions:
 name:pick-up
  params:['?x-object', '?y']
  precond:(is-red (color ?x))
  effect:(if (is-blue ?y) (and (assign (is-blue ?x) 1.0) (assign (is-red ?y) 0.0)))
{'is-green-f': None}


In [3]:
"""Create the base model, executor and more"""
config.conept_dim = 100
config.perception_model_name = "SetNet"
config.channel_dim = 2
model = KaranirThanagor(domain, config)
#model.print_summary()

from karanir.thanagor.dsl.vqa_primitives import *


In [4]:
from karanir.thanagor.model import fit_scenes, generate_grounding, generate_scenes
concept_dim = config.concept_dim


train_scenes = generate_scenes(domain, 32)
train_scenes = generate_grounding(domain, train_scenes)

test_scenes = generate_scenes(domain, 164)
test_scenes = generate_grounding(domain, test_scenes)

#print(demo_scenes)
print(evaluate_scenes(test_scenes, model))
fit_scenes(train_scenes, model, epochs = 500, lr = 1e-3, batch_size = 4)
print(evaluate_scenes(test_scenes, model))


loss:102.864 acc:0.5990853658536586[393/656]
0.5990853658536586
epoch:118 loss:0.021 acc:1.0[16/16]6]

KeyboardInterrupt: 

In [ ]:
"""
history_prev = []
history_post = []
import tqdm

for i in tqdm.tqdm(range(1,8 + 1,1)):
    train_scenes = generate_scenes(domain, i)
    train_scenes = generate_grounding(domain, train_scenes)

    test_scenes = generate_scenes(domain, 164)
    test_scenes = generate_grounding(domain, test_scenes)
    '''Generate samples'''

    model = KaranirThanagor(domain, config)
    prev = evaluate_scenes(test_scenes, model, False)
    fit_scenes(train_scenes, model, epochs = 1000, lr = 1e-3, batch_size = 4, verbose = False)
    post = evaluate_scenes(test_scenes, model, False)
    history_prev.append(prev)
    history_post.append(post)
import matplotlib.pyplot as plt
plt.plot(history_prev)
plt.plot(history_post)
plt.show()
"""

"\nhistory_prev = []\nhistory_post = []\nimport tqdm\n\nfor i in tqdm.tqdm(range(1,8 + 1,1)):\n    train_scenes = generate_scenes(domain, i)\n    train_scenes = generate_grounding(domain, train_scenes)\n\n    test_scenes = generate_scenes(domain, 164)\n    test_scenes = generate_grounding(domain, test_scenes)\n    '''Generate samples'''\n\n    model = KaranirThanagor(domain, config)\n    prev = evaluate_scenes(test_scenes, model, False)\n    fit_scenes(train_scenes, model, epochs = 1000, lr = 1e-3, batch_size = 4, verbose = False)\n    post = evaluate_scenes(test_scenes, model, False)\n    history_prev.append(prev)\n    history_post.append(post)\nimport matplotlib.pyplot as plt\nplt.plot(history_prev)\nplt.plot(history_post)\nplt.show()\n"